<a href="https://colab.research.google.com/github/june-oh/cseg109/blob/main/LAB6_Fastspeech2_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 : torch를 이용한  transformer 구현

- https://nlp.seas.harvard.edu/annotated-transformer/



# Lab6 Fast Speech 2 Train  & Inference

- Fast Speech 2
  - https://arxiv.org/abs/2006.04558
- VocGan
  - https://arxiv.org/abs/2007.15256
- Fast Speech 2 Korean (code, pretrained model)
  - https://github.com/HGU-DLLAB/Korean-FastSpeech2-Pytorch.git


In [ ]:
!rm -rf ./* .config && ls -alh

total 12K
drwxr-xr-x 1 root root 4.0K Dec  4 14:16 .
drwxr-xr-x 1 root root 4.0K Dec  4 14:14 ..


In [ ]:
!pip install -q tgt g2pk pyworld librosa==0.8.0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 14.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/HGU-DLLAB/Korean-FastSpeech2-Pytorch.git .

Cloning into '.'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 323 (delta 17), reused 19 (delta 8), pack-reused 286
Receiving objects: 100% (323/323), 11.43 MiB | 20.11 MiB/s, done.
Resolving deltas: 100% (171/171), done.


edit `hparams.py`

In [ ]:
!pip install -q kaggle

## Upload Kaggle Token
<img src="https://github.com/june-oh/cseg109/blob/main/img/token.png?raw=true" width=70%>

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"teacody","key":"4ada820bda1852ce79416cb53b642faf"}'}

In [ ]:
!mkdir ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

## Download KSS dataset
- https://www.kaggle.com/datasets/bryanpark/korean-single-speaker-speech-dataset

In [ ]:
!kaggle datasets download bryanpark/korean-single-speaker-speech-dataset

 99% 2.84G/2.86G [00:21<00:00, 146MB/s]
100% 2.86G/2.86G [00:21<00:00, 144MB/s]


In [ ]:
!unzip -q  korean-single-speaker-speech-dataset

In [ ]:
!ls

assets		korean-single-speaker-speech-dataset.zip  preprocessed	    transcript.v.1.4.txt
audio		kss					  preprocess.py     transformer
data		LICENSE					  README.md	    utils.py
dataset.py	loss.py					  requirements.txt  vocoder
evaluate.py	modules.py				  synthesize.py
fastspeech2.py	optimizer.py				  text
hparams.py	prepare_align.py			  train.py


In [ ]:
!ls

assets		korean-single-speaker-speech-dataset.zip  preprocessed	    transcript.v.1.4.txt
audio		kss					  preprocess.py     transformer
data		LICENSE					  README.md	    utils.py
dataset.py	loss.py					  requirements.txt  vocoder
evaluate.py	modules.py				  synthesize.py
fastspeech2.py	optimizer.py				  text
hparams.py	prepare_align.py			  train.py


In [ ]:
import gdown
url = "https://drive.google.com/uc?id=1LgZPfWAvPcdOpGBSncvMgv54rGIf1y-H"
gdown.download(url)

Downloading...
From: https://drive.google.com/uc?id=1LgZPfWAvPcdOpGBSncvMgv54rGIf1y-H
To: /content/TextGrid.zip
100%|██████████| 11.1M/11.1M [00:00<00:00, 39.1MB/s]


'TextGrid.zip'

## preprocessing

In [ ]:
!python3 preprocess.py

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  libavdevice    58. 13.100 / 58. 13.100
  libavfilter     7.110.100 /  7.110.100
  libswscale      5.  9.100 /  5.  9.100
  libswresample   3.  9.100 /  3.  9.100
  libpostproc    55.  9.100 / 55.  9.100
Guessed Channel Layout for Input Stream #0.0 : mono
Input #0, wav, from './kss/wavs_bak/3/3_2493.wav':
  Metadata:
    encoded_by      : Pro Tools
    originator_reference: uCvJKtXZKNeaaaGk
    date            : 2018-03-25
    creation_time   : 20:19:19
    time_reference  : 918532612
  Duration: 00:00:01.67, bitrate: 753 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, mono, s16, 705 kb/s
Stream mapping:
  Stream #0:0 -> #0:0 (pcm_s16le (native) -> pcm_s16le (native))
Press [q] to stop, [?] for help
Output #0, wav, to './kss/wavs/3_2493.wav':
  Metadata:
    ITCH            : Pro Tools
    originator_reference: uCvJKtXZKNeaaaGk
    ICRD            : 2018-03-25
    time_reference  : 918532612
    ISFT            : Lavf58.76.10

In [ ]:
url = 'https://drive.google.com/uc?id=1GxaLlTrEhq0aXFvd_X1f4b-ev7-FH8RB'
gdown.download(url,"./vocoder/pretrained_models/")

make log dir

In [ ]:
!mkdir -p log/kss

load tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir log/kss

Edit train.py

start train loop

In [ ]:
!python3 train.py